<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=af87eb11730ec9da0a9f24213bf4530e2275be47f71fb151992b5f5d57ac0a95
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-31 11:48:05
-------------------
qualified stocks: 88
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.37 K
Current:  1.43 C
-------------------
Today PnL: 97.25 K (0.69%)
Current PnL: -26.27 L (-16.75%)
CY Booked + Current PnL: -11.98 L (-7.64%)
-------------------
Total profit:  1.46 L
Total loss:  -27.73 L
-------------------
Total Booked + Current PnL: 15.00 L (11.55%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.01%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 91.65 L (64.19%)
Deployed:  1.30 C
Current:  1.43 C
CAGR/XIRR %: 5.26%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.00,0.93,-19.65,24.57,0.09,19896.0,-19799.0,80978.0,88.05,39.0,M-SC,3.18,253.0,-1.00,0.57,6.39,OX40N,NTT,DURABLES
53,NESTLEIND,1377.00,0.58,12.98,7.45,21.40,22340.0,34445.0,299871.0,4.80,64.0,X-LC,5.19,12.0,1.54,2.10,20.13,XY25,NTT,FMCG
51,MASFIN,398.61,2.35,-1.45,23.85,22.05,23028.0,-1425.0,96555.0,-15.09,55.0,H-SC,6.58,164.0,-0.06,0.68,40.35,XR,ATH,FINANCE
84,VOLTAS,1530.00,0.53,6.22,12.68,19.69,25825.0,11928.0,203670.0,-4.12,46.0,X-MC,2.12,78.0,0.46,1.43,13.44,XY25,NTT,AC
44,ITC,452.00,0.65,-1.18,12.10,10.78,28736.0,-2833.0,237485.0,-41.15,49.0,X-LC,1.99,5.0,-0.10,1.66,4.00,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,730.66,3.75,-1.05,48.51,46.95,60145.0,-1313.0,123984.0,-8.38,67.0,H-SC,9.47,154.0,-0.02,0.87,55.32,MH,ATH,POWER
10,BAJAJHLDNG,14451.52,3.53,2.30,27.10,30.03,36975.0,3070.0,136440.0,-0.67,51.0,X-LC,12.15,35.0,0.08,0.96,11.00,X40,ATH,FINANCE
76,TITAGARH,1548.00,2.66,-15.09,71.64,45.74,147315.0,-36544.0,205633.0,13.22,66.0,H-SC,4.81,174.0,-0.25,1.44,33.74,XY24,NTT,ENGINEERING
72,SYMPHONY,1306.00,2.63,-32.18,47.40,-0.03,55019.0,-55068.0,116073.0,-45.51,59.0,M-SC,16.12,206.0,-1.00,0.81,5.48,OX40N,NTT,DURABLES
0,5PAISA,593.00,2.53,-35.25,74.36,12.90,105206.0,-77014.0,141482.0,107.93,60.0,H-SC,12.26,161.0,-0.73,0.99,17.64,OX40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJESHEXPO,518.00,-4.66,-64.63,182.89,0.07,88746.0,-88653.0,48524.0,1679.04,42.0,L-SC,4.34,268.0,-1.00,0.34,20.68,OX40N,NTT,JEWELLERY
19,COFFEEDAY,80.00,-2.21,-38.05,118.34,35.27,83251.0,-43200.0,70349.0,-51.71,54.0,L-SC,7.03,270.0,-0.52,0.49,72.18,XR,NTT,HOTELS
39,INDIAMART,4810.62,-1.64,-5.03,117.67,106.72,137833.0,-6201.0,117135.0,-52.92,41.0,H-SC,5.88,139.0,-0.04,0.82,17.18,AR,ATH,MISC
56,QUESS,424.00,-1.54,-30.24,104.77,42.84,47509.0,-19660.0,45346.0,-54.27,42.0,X-SC,31.61,83.0,-0.41,0.32,2.59,XY24,NTT,MISC
75,TCS,4389.97,-0.86,-11.49,36.38,20.71,121788.0,-43471.0,334766.0,-23.62,49.0,X-LC,3.03,1.0,-0.36,2.34,11.86,X40,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1641.55,1.31,-6.10,121.41,107.91,168325.0,-9002.0,138642.0,-23.39,42.0,M-SC,11.64,216.0,-0.05,0.97,3.37,OX40N,ATH,CABLES
40,INDIGOPNTS,1408.00,1.89,-18.76,23.11,0.02,32774.0,-32740.0,141819.0,116.52,39.0,M-SC,8.36,234.0,-1.00,0.99,23.36,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,0.93,-19.65,24.57,0.09,19896.0,-19799.0,80978.0,88.05,39.0,M-SC,3.18,253.0,-1.00,0.57,6.39,OX40N,NTT,DURABLES
48,KANSAINER,340.00,0.96,-21.20,44.01,13.47,93517.0,-57177.0,212490.0,-67.53,55.0,H-SC,6.49,158.0,-0.61,1.49,8.05,XY24,NTT,PAINTS
68,SIS,528.00,0.00,-23.38,58.27,21.26,49569.0,-25964.0,85068.0,2007.77,52.0,H-SC,4.48,166.0,-0.52,0.60,14.94,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.0,-0.21,7.91,59.64,72.27,109055.0,13407.0,182855.0,-7.44,56.0,M-LC,5.23,99.0,0.12,1.28,14.58,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.00,-0.21,7.91,59.64,72.27,109055.0,13407.0,182855.0,-7.44,56.0,M-LC,5.23,99.0,0.12,1.28,14.58,XR,NTT,IT
6,ASIANTILES,137.00,0.07,-1.01,83.52,81.67,77561.0,-945.0,92865.0,8361.11,66.0,L-SC,16.00,271.0,-0.01,0.65,79.84,XR,NTT,CERAMICS
49,KPIGREEN,730.66,3.75,-1.05,48.51,46.95,60145.0,-1313.0,123984.0,-8.38,67.0,H-SC,9.47,154.0,-0.02,0.87,55.32,MH,ATH,POWER
39,INDIAMART,4810.62,-1.64,-5.03,117.67,106.72,137833.0,-6201.0,117135.0,-52.92,41.0,H-SC,5.88,139.0,-0.04,0.82,17.18,AR,ATH,MISC
26,FINCABLES,1641.55,1.31,-6.10,121.41,107.91,168325.0,-9002.0,138642.0,-23.39,42.0,M-SC,11.64,216.0,-0.05,0.97,3.37,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.12,-19.43,92.96,55.47,182221.0,-47265.0,196021.0,-28.68,26.0,X-MC,16.47,64.0,-0.26,1.37,0.44,X40N,ATH,FINANCE
23,DIXON,18931.72,0.94,-20.90,59.38,26.07,91691.0,-40807.0,154414.0,-61.53,26.0,X-MC,16.15,56.0,-0.45,1.08,0.94,X40N,ATH,IT
28,GLAXO,3466.20,0.20,-4.10,45.10,39.15,86192.0,-8172.0,191112.0,-22.92,29.0,X-MC,10.88,60.0,-0.09,1.34,23.71,X40,ATH,PHARMA
35,HONAUT,58357.33,0.02,-18.95,76.84,43.32,101429.0,-30872.0,132000.0,-30.40,29.0,X-SC,7.22,90.0,-0.30,0.92,1.35,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-0.01,-27.22,125.53,64.15,217405.0,-64761.0,173190.0,-57.62,33.0,X-SC,5.66,82.0,-0.30,1.21,0.39,XY24,BTT,CEMENT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.5,1.45,-17.42,52.00,25.51,79908.0,-32425.0,153670.0,-5.62,42.0,H-LC,0.99,49.0,-0.41,1.08,13.24,AR,ATH,ELECTRICAL
80,UNITDSPR,1644.0,-0.27,4.31,15.77,20.77,38517.0,10096.0,244240.0,-4.74,48.0,X-MC,1.50,62.0,0.26,1.71,11.53,X40N,NTT,BREWERIES
13,BERGEPAINT,680.0,0.88,-5.55,28.25,21.14,60662.0,-12608.0,214731.0,-12.25,40.0,X-MC,1.65,74.0,-0.21,1.50,19.25,XY24,NTT,PAINTS
21,DABUR,735.0,2.16,-1.00,45.16,43.70,110904.0,-2488.0,245580.0,-9.24,58.0,X-MC,1.99,72.0,-0.02,1.72,13.90,XY24,BTT,FMCG
44,ITC,452.0,0.65,-1.18,12.10,10.78,28736.0,-2833.0,237485.0,-41.15,49.0,X-LC,1.99,5.0,-0.10,1.66,4.00,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.01,-27.22,125.53,64.15,217405.0,-64761.0,173190.0,-57.62,33.0,X-SC,5.66,82.0,-0.30,1.21,0.39,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-0.12,-19.43,92.96,55.47,182221.0,-47265.0,196021.0,-28.68,26.0,X-MC,16.47,64.0,-0.26,1.37,0.44,X40N,ATH,FINANCE
12,BATAINDIA,2096.00,0.56,-39.15,122.19,35.20,95672.0,-50372.0,78298.0,2.67,36.0,X-SC,17.36,92.0,-0.53,0.55,0.57,X40,NTT,FOOTWEAR
20,COLPAL,3726.84,0.65,-21.52,80.32,41.51,166005.0,-56685.0,206680.0,-6.53,35.0,X-MC,8.39,61.0,-0.34,1.45,0.65,XY25,ATH,FMCG
54,PAGEIND,51605.08,0.76,-11.04,44.01,28.11,78855.0,-22235.0,179175.0,-35.49,34.0,X-MC,15.20,55.0,-0.28,1.26,0.76,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.54,-30.24,104.77,42.84,47509.0,-19660.0,45346.0,-54.27,42.0,X-SC,31.61,83.0,-0.41,0.32,2.59,XY24,NTT,MISC
60,RELAXO,1176.00,1.07,-47.59,190.59,52.31,145008.0,-69076.0,76084.0,-44.25,46.0,X-SC,7.38,91.0,-0.48,0.53,1.84,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,0.56,-39.15,122.19,35.20,95672.0,-50372.0,78298.0,2.67,36.0,X-SC,17.36,92.0,-0.53,0.55,0.57,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.52,-3.37,28.40,24.08,34839.0,-4276.0,122674.0,-11.26,41.0,X-SC,7.71,89.0,-0.12,0.86,15.59,XY24,NTT,HEALTHCARE
35,HONAUT,58357.33,0.02,-18.95,76.84,43.32,101429.0,-30872.0,132000.0,-30.40,29.0,X-SC,7.22,90.0,-0.30,0.92,1.35,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-0.86,-11.49,36.38,20.71,121788.0,-43471.0,334766.0,-23.62,49.0,X-LC,3.03,1.0,-0.36,2.34,11.86,X40,ATH,IT
42,INFY,1972.00,-0.33,10.03,22.01,34.24,75774.0,31384.0,344272.0,-14.21,52.0,X-LC,7.32,2.0,0.41,2.41,19.14,X40,NTT,IT
77,TMPV,600.00,1.16,-24.99,63.96,22.98,108371.0,-56463.0,169435.0,-21.86,57.0,X-LC,5.85,3.0,-0.52,1.19,6.57,XY24,NTT,AUTO
83,VBL,671.64,0.36,-1.79,38.14,35.67,118309.0,-5646.0,310196.0,-13.42,60.0,X-LC,2.92,4.0,-0.05,2.17,11.82,X40N,ATH,FMCG
44,ITC,452.00,0.65,-1.18,12.10,10.78,28736.0,-2833.0,237485.0,-41.15,49.0,X-LC,1.99,5.0,-0.10,1.66,4.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1800.00,1.63,-36.86,110.48,32.89,53855.0,-28460.0,48746.0,-701.92,57.0,L-MC,6.63,259.0,-0.53,0.34,34.24,XR,NTT,BANKS
6,ASIANTILES,137.00,0.07,-1.01,83.52,81.67,77561.0,-945.0,92865.0,8361.11,66.0,L-SC,16.00,271.0,-0.01,0.65,79.84,XR,NTT,CERAMICS
51,MASFIN,398.61,2.35,-1.45,23.85,22.05,23028.0,-1425.0,96555.0,-15.09,55.0,H-SC,6.58,164.0,-0.06,0.68,40.35,XR,ATH,FINANCE
14,BSOFT,831.70,1.27,-22.22,93.87,50.79,102689.0,-31258.0,109395.0,-0.27,53.0,H-SC,11.10,151.0,-0.30,0.77,27.37,XR,ATH,IT
49,KPIGREEN,730.66,3.75,-1.05,48.51,46.95,60145.0,-1313.0,123984.0,-8.38,67.0,H-SC,9.47,154.0,-0.02,0.87,55.32,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.07,-1.01,83.52,81.67,77561.0,-945.0,92865.0,8361.11,66.0,L-SC,16.00,271.0,-0.01,0.65,79.84,XR,NTT,CERAMICS
49,KPIGREEN,730.66,3.75,-1.05,48.51,46.95,60145.0,-1313.0,123984.0,-8.38,67.0,H-SC,9.47,154.0,-0.02,0.87,55.32,MH,ATH,POWER
14,BSOFT,831.70,1.27,-22.22,93.87,50.79,102689.0,-31258.0,109395.0,-0.27,53.0,H-SC,11.10,151.0,-0.30,0.77,27.37,XR,ATH,IT
76,TITAGARH,1548.00,2.66,-15.09,71.64,45.74,147315.0,-36544.0,205633.0,13.22,66.0,H-SC,4.81,174.0,-0.25,1.44,33.74,XY24,NTT,ENGINEERING
37,ICICIPRULI,790.00,1.52,10.01,18.83,30.73,40809.0,19720.0,216725.0,-15.76,68.0,X-MC,4.28,75.0,0.48,1.52,24.06,X40,ATH,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.94
1,25,44.10
2,50,75.11


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.19
MC    30.16
LC    25.64
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.49
X40      23.49
X40N     12.59
XY25      9.96
XR        9.63
AR        8.62
OX40N     7.36
SR        0.98
MH        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.71
H-SC    23.58
X-LC    22.24
M-SC    11.29
X-SC     7.84
H-MC     4.72
L-SC     1.48
M-MC     1.39
M-LC     1.28
H-LC     1.08
L-LC     1.04
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.10,-6.86,41.22
IT,12.49,-17.72,76.92
FINANCE,10.49,-15.28,64.22
MISC,7.02,-29.76,82.84
ELECTRICAL,5.84,-12.68,53.95
PAINTS,5.69,-13.48,30.33
INSURANCE,4.69,-1.49,36.22
PHARMA,3.96,-4.00,36.73
AUTO,2.82,-30.29,75.46


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3140699.0,21
AR,1340009.0,10
XR,1286087.0,13
X40,1043448.0,15
X40N,887137.0,9
OX40N,740414.0,10
XY25,386098.0,7
SR,280824.0,2
MH,60145.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3610709.0,25
M-SC,1446036.0,15
X-MC,1443618.0,16
X-LC,889217.0,13
X-SC,799395.0,8
H-MC,405533.0,3
L-SC,249558.0,3
M-LC,109055.0,1
H-LC,79908.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1240998.0      6
           AR         917060.0      5
M-SC       XY24       816100.0      6
H-SC       XR         783830.0      7
X-MC       X40        489635.0      7
           XY24       406583.0      3
X-LC       X40        370312.0      6
X-MC       X40N       355570.0      4
H-SC       OX40N      327852.0      4
M-SC       OX40N      323816.0      5
X-SC       X40N       316141.0      3
           XY24       299753.0      3
H-SC       SR         280824.0      2
H-MC       AR         215456.0      2
X-LC       X40N       215426.0      2
X-MC       XY25       191830.0      2
H-MC       XY24       190077.0      1
X-LC       XY24       187188.0      2
X-SC       X40        183501.0      2
M-SC       XR         178535.0      2
L-SC       XR         160812.0      2
M-SC       AR         127585.0      2
X-LC       XY25       116291.0      3
M-LC       XR         109055.0      1
L-SC       OX40N       88746.0      1
H-LC       AR          79908.0      1
H-SC       MH          60145.0      1
L-MC       XR          53855.0      1
M-MC       XY25        47348.0      1
L-LC       XY25        30629.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
